### Deep Neural Network for MNIST Classification

In [3]:
import numpy as np
import tensorflow as tf
import time
# Imports the test and validation data from this below import
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Outline the model

In [25]:
input_size = 784
output_size = 10
hidden_layer_size = 100

tf.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.float32, [None, output_size])


weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])
outputs_1 = tf.nn.relu(tf.matmul(inputs,weights_1) + biases_1)


weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])
outputs_2 = tf.nn.relu(tf.matmul(outputs_1, weights_2) + biases_2)

weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", [output_size])
outputs = tf.matmul(outputs_2,weights_3) + biases_3
loss = tf.nn.softmax_cross_entropy_with_logits(logits = outputs, labels = targets)


mean_loss = tf.reduce_mean(loss)
optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)
out_equals_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))


session = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
session.run(initializer)


batch_size = 100
batches_number = mnist.train._num_examples // batch_size


max_epochs = 15

previous_validation_loss = 9999999.
start_time = time.time()    

for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0.
    
    for batch_counter in range(batches_number):
         
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        _, batch_loss = session.run([optimize, mean_loss],
                                   feed_dict={inputs: input_batch, targets: target_batch})
        curr_epoch_loss += batch_loss
        
    curr_epoch_loss /= batches_number
    
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    
    validation_loss, validation_accuracy = session.run([mean_loss, accuracy],
                                                   feed_dict={inputs: input_batch, targets: target_batch})
    
    print('Epoch ' + str(epoch_counter + 1) +
         '. Training loss: ' + '{0:.3f}'.format(curr_epoch_loss) +
         '. Validation loss: ' + '{0:.3f}'.format(validation_loss) +
         '. Validation accuracy: ' + '{0:.2f}'.format(validation_accuracy * 100) + '%')
    
    if validation_loss > previous_validation_loss:
        break
    previous_validation_loss = validation_loss

print('End of training')
print("Training time: %s seconds" % (time.time() - start_time))

D:\AnacondaIDE\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 1. Training loss: 0.330. Validation loss: 0.153. Validation accuracy: 95.54%
Epoch 2. Training loss: 0.138. Validation loss: 0.111. Validation accuracy: 96.78%
Epoch 3. Training loss: 0.097. Validation loss: 0.092. Validation accuracy: 97.56%
Epoch 4. Training loss: 0.075. Validation loss: 0.086. Validation accuracy: 97.38%
Epoch 5. Training loss: 0.059. Validation loss: 0.083. Validation accuracy: 97.60%
Epoch 6. Training loss: 0.049. Validation loss: 0.080. Validation accuracy: 97.60%
Epoch 7. Training loss: 0.040. Validation loss: 0.079. Validation accuracy: 97.70%
Epoch 8. Training loss: 0.032. Validation loss: 0.081. Validation accuracy: 97.84%
End of training
Training time: 8.464206457138062 seconds


### Test the data

In [11]:
input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)
test_accuracy = session.run([accuracy],
                           feed_dict={inputs: input_batch, targets: target_batch})

test_accuracy_percent = test_accuracy[0] * 100.

print('Test accuracy: ' + '{0:.2f}'.format(test_accuracy_percent) + '%')

Test accuracy: 97.49%
